In [1]:
import pandas as pd
import numpy as np
import csv
from math import sqrt

In [2]:
#loading data into dictionary : train_data[u_id][j_id]=rating
#rating is from -10 to 10.
#Loading rating in range [0,20] by adding 10 to eliminate negative sqrt 
def load_data():
    global n_users
    n_users = 0
    train_data = {}
    f = csv.DictReader(open("train/train.csv"))
    ct = 0
    for row in f:
        ct += 1
        u_id = int(row['user_id'])
        n_users = max(n_users,u_id)
        j_id = int(row['joke_id'])
        rating = float(row['Rating']) + 10.0
        if u_id in train_data:
            current_ratings = train_data[u_id]
        else:
            current_ratings = {}
        current_ratings[j_id] = rating
        train_data[u_id] = current_ratings
    print("no. of ratings: "+str(ct))    
    return train_data

In [3]:
train_dict = load_data()

no. of ratings: 1092059


In [4]:
print(n_users)
train_dict[1]

40863


{1: 10.219,
 2: 0.7189999999999994,
 4: 3.2189999999999985,
 5: 10.875,
 6: 0.3439999999999994,
 7: 0.9689999999999994,
 8: 2.5310000000000015,
 9: 1.2810000000000006,
 12: 1.2189999999999994,
 17: 18.781,
 19: 18.781,
 21: 18.781,
 24: 9.75,
 25: 18.781,
 26: 18.781,
 32: 10.062,
 39: 10.062,
 42: 10.062,
 43: 18.781,
 44: 18.688,
 51: 10.031,
 52: 10.219,
 56: 18.688,
 59: 18.688,
 62: 18.781,
 66: 19.311999999999998,
 70: 10.062,
 79: 19.811999999999998,
 81: 18.781,
 82: 13.625,
 83: 19.311999999999998,
 92: 10.75,
 93: 5.0,
 95: 12.0,
 97: 12.031,
 98: 15.687999999999999,
 99: 19.656,
 108: 10.0,
 109: 18.781,
 110: 18.781,
 117: 18.688,
 118: 10.0}

In [5]:
#indexing is 1-based in dataset
n_items = 139

def pcs(rating1, rating2):  #pass pcs(train_dict[uid_1],train_dict[uid_2])
        sum_xy = 0
        sum_x = 0
        sum_y = 0
        sum_x2 = 0
        sum_y2 = 0
        n = 0
        for key in rating1:
            if key in rating2:
                n += 1
                x = rating1[key]
                y = rating2[key]
                sum_xy += x * y
                sum_x += x
                sum_y += y
                sum_x2 += pow(x, 2)
                sum_y2 += pow(y, 2)
        if n == 0:
            return 0
        # denominator
#         print(sqrt(sum_x2 - pow(sum_x, 2) / n))
#         print(sqrt(sum_y2 - pow(sum_y, 2) / n))
        denominator = (sqrt(abs(sum_x2 - pow(sum_x, 2) / n))* sqrt(abs(sum_y2 - pow(sum_y, 2) / n)))
        if denominator == 0:
            return 0
        else:
            return (sum_xy - (sum_x * sum_y) / n) / denominator
        

In [6]:
neigh_dict = {}
def neighbors(u_id):
        """creates a sorted list of users based on their distance to
        user-id u_id"""
        if u_id in neigh_dict:
            return neigh_dict[u_id]
        else:
            distances = []   #list of (u_id,distance)
            for instance in train_dict:
                if instance != u_id:
                    distance = pcs(train_dict[u_id],train_dict[instance])
                    distances.append((instance, distance))
            # sort based on distance -- closest first
            distances.sort(key=lambda artistTuple: artistTuple[1],reverse=True)
            neigh_dict[u_id] = distances
            return neigh_dict[u_id]

In [11]:
import sys
import time
def predict(u_id,j_id):
    global ct
    ct = 0
    neigh_list = neighbors(u_id)
    k = 5 # 100 nearest neighbors
    ratings_sum = 0.
    ratings_ct = 0.
    k_neigh_list = neigh_list[:k]
    for (uid,dist) in k_neigh_list:
        if j_id in train_dict[uid]:
            ratings_sum += train_dict[uid][j_id]
            ratings_ct += 1.0
    sys.stdout.write(str(ct))
    ct += 1
    sys.stdout.flush()
    time.sleep(0.5)        
    if ratings_ct == 0.0:
        return 0.0    #or retun average of user u_id
    else:
        return ratings_sum/ratings_ct - 10.0      
    
    

In [12]:
# print(neighbors(1)[:10])
predict(1,3)

0

1.179499999999999

In [13]:
test_df = pd.read_csv("test.csv")
test_df.head(5)

,id,user_id,joke_id
0,6194_11,6194,11
1,19356_3,19356,3
2,23426_79,23426,79
3,40030_3,40030,3
4,19806_115,19806,115


In [14]:
def label_rating(row):
    return predict(int(row['user_id']),int(row['joke_id']))
    

In [ ]:
test_df['Rating'] = test_df.apply (lambda row: label_rating (row),axis=1)

0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [ ]:
test_df.head(5)